# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ramav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ramav\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///../'+'data/DisasterResponse.db')
df = pd.read_sql_table('message_categories', con=engine)
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [5]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    '''
    To convert a string of text into list of tokens by:
    1. Capitalization removal, punctuation removeal and stop word removal
    2. Split into a list of words
    3. Reduce the words to their root forms by lemmitization 
    
    Args:
    text: String to be tokenized
    
    Returns:
    clean_tokens: Tokenized data
    '''
    #Replace anything that isn't A-Z or )-9 with a space. 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    # Split text into words using NLTK
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(t) for t in tokens]

    # Lemmatize verbs by specifying pos
    cleaned_tokens = [WordNetLemmatizer().lemmatize(t, pos='v') for t in lemmed]

    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000199294506A8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
print("Baseline results:")
print(classification_report(y_test, y_pred, target_names=y.columns))

Baseline results:
                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4993
               request       0.82      0.49      0.62      1126
                 offer       0.00      0.00      0.00        27
           aid_related       0.78      0.69      0.73      2723
          medical_help       0.68      0.05      0.09       533
      medical_products       0.79      0.07      0.13       327
     search_and_rescue       0.67      0.02      0.04       174
              security       1.00      0.01      0.02       123
              military       0.80      0.06      0.10       215
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.41      0.56       411
                  food       0.89      0.55      0.68       760
               shelter       0.79      0.38      0.52       577
              clothing       1.00      0.07      0.14        94
                 mone

C:\Users\ramav\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ramav\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ramav\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ramav\anaconda3\envs\gpu

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000199294506A8>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000199294506A8>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [22]:
parameters = {
    'vect__ngram_range' : ((1,1), (1,2)),
    'tfidf__smooth_idf' : [True, False],
    'clf__estimator__max_depth' : [40, 50]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000199294506A8>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [40, 50],
                         'tfidf__smooth_idf': [True, False],
                         'vect__ngram_range': ((1, 1), (1, 2))})

In [25]:
y_pred2 = cv.predict(X_test)

In [26]:
print("Best Parameters for classifier:",cv.best_params_)

Best Parameters for classifier: {'clf__estimator__max_depth': 50, 'tfidf__smooth_idf': False, 'vect__ngram_range': (1, 2)}


In [27]:
print("Results after parameter tuning:")
print(classification_report(y_test, y_pred2, target_names=y.columns))

Results after parameter tuning:
                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      4993
               request       0.95      0.09      0.17      1126
                 offer       0.00      0.00      0.00        27
           aid_related       0.88      0.35      0.50      2723
          medical_help       0.00      0.00      0.00       533
      medical_products       0.67      0.01      0.01       327
     search_and_rescue       0.00      0.00      0.00       174
              security       0.00      0.00      0.00       123
              military       0.00      0.00      0.00       215
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       411
                  food       0.86      0.01      0.02       760
               shelter       0.00      0.00      0.00       577
              clothing       0.00      0.00      0.00        94
       

C:\Users\ramav\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ramav\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ramav\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ramav\anaconda3\envs\gpu

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [28]:
#As stopwords were removed as part of tokenization, count stopwords to bring in that information
class StopwordCounter(BaseEstimator, TransformerMixin):

    def count_stopwords(self, text):
        text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
        # Split text into words using NLTK
        tokens = word_tokenize(text)
        #Filter for stopwords
        stopwords_in_text = [w for w in tokens if w in set(stopwords.words('english'))]
        return len(stopwords_in_text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_stopword_count = pd.Series(X).apply(self.count_stopwords)
        return pd.DataFrame(X_stopword_count)

In [36]:
pipeline2 = Pipeline([
                ('features', FeatureUnion([
                    ('tokenize_pipeline', Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer())
                    ])),
                    
                    ('stopword_count', StopwordCounter())
                ])),
    
                ('clf', MultiOutputClassifier(DecisionTreeClassifier(max_depth=[30])))
            ])

In [37]:
parameters2 = {
    'features__tokenize_pipeline__vect__ngram_range' : ((1,1), (1,2)),
    'features__tokenize_pipeline__tfidf__smooth_idf' : [True, False],
    'clf__estimator__max_depth' : ['gini','log_loss','entropy']
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

In [ ]:
cv2.fit(X_train, y_train)
y_pred3 = cv2.predict(X_test)

print("Best Parameters for Decision Tree Classifier:",cv2.best_params_)
print("Results after parameter tuning:")
print(classification_report(y_test, y_pred3, target_names=Y.columns))
# This leads to lower overall precision, so better to use the original model.

### 9. Export your model as a pickle file

In [40]:
model_filepath='models/classifier.pkl'
pickle.dump(cv, open('../'+model_filepath,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.